In [ ]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
                                  
!cp kaggle.json ~/.kaggle/      #   (!cp [   API token file name   ] ~/.kaggle/)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sanikamal/horses-or-humans-dataset

100% 306M/307M [00:02<00:00, 139MB/s]
100% 307M/307M [00:02<00:00, 140MB/s]


In [ ]:
from zipfile import ZipFile
zf = ZipFile('horses-or-humans-dataset.zip', 'r')
zf.extractall('/content')
zf.close()

In [ ]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [ ]:
from keras.layers import Input,Dense,Lambda,Flatten
from keras.models import Model,Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import glob
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB7,preprocess_input

In [ ]:
IMAGE_SIZE=[224,224,3]

In [ ]:
effb7=EfficientNetB7(include_top=False,input_shape=IMAGE_SIZE,weights='imagenet')
effb7.summary()

258080768/258076736 [==============================] - 2s 0us/step
Model: "efficientnetb7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
__________________

In [ ]:
for layer in effb7.layers:
  layer.trainable=False

In [ ]:
x=Flatten()(effb7.output)
prediction=Dense(1,activation='sigmoid')(x)
model=Model(inputs=effb7.input,outputs=prediction)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
____________________________________________________________________________________________

In [ ]:
from keras.optimizers import Adam
adam=Adam()
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

In [ ]:
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

)

In [ ]:
test_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'

)

In [ ]:
train_path='horse-or-human/train'
test_path='horse-or-human/validation'

In [ ]:
train_set=train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=32,class_mode='categorical')

Found 1027 images belonging to 2 classes.


In [ ]:
test_set=test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,class_mode='categorical')

Found 256 images belonging to 2 classes.


In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint=ModelCheckpoint(filepath='effb7.h5',verbose=2,save_best_only=True)
callbacks=[checkpoint]
start=datetime.now()
model_history=model.fit_generator(train_set,validation_data=test_set,epochs=5,steps_per_epoch=15,validation_steps=32,callbacks=callbacks,verbose=1)

duration=datetime.now()-start

print('Total elapsed time : ',duration)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
15/15 [==============================] - 33s 1s/step - loss: 2.1395 - accuracy: 0.5000 - val_loss: 1.5405 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 1.54047, saving model to effb7.h5
Epoch 2/5
15/15 [==============================] - 9s 602ms/step - loss: 1.7024 - accuracy: 0.5000
Epoch 3/5
15/15 [==============================] - 9s 587ms/step - loss: 1.9437 - accuracy: 0.5000
Epoch 4/5
15/15 [==============================] - 9s 597ms/step - loss: 1.6428 - accuracy: 0.5000
Epoch 5/5
15/15 [==============================] - 9s 569ms/step - loss: 1.7680 - accuracy: 0.5000
Total elapsed time :  0:01:12.217652
